In [1]:
import json
import os
import pickle
import torch
from torch.utils.data import DataLoader
from pathlib import Path
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
from transformers import BertTokenizer, BertForSequenceClassification, AdamW

In [2]:
SPLIT_SAMPLES = 8000
DATASET_NAME = "IMDb"
DATASET_SMALLNAME = "aclImdb"
os.environ["CUDA_VISIBLE_DEVICES"]="1"
REFORMED_DATASET_PATH = f"dataset/{DATASET_NAME}/original_augmented_1x_{DATASET_SMALLNAME}"
OUTPUT_PATH = f"checkpoints/{DATASET_NAME}/original_augmented_1x_output_scheduling_warmup_bs8_2"
REPS_PATH = "reps"
PICKLE_PATH = f"dataset/{DATASET_NAME}/cf_augmented_examples"
TRAIN_SPLIT = "train"
TEST_SPLIT = "test"

In [3]:
class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [4]:
# Load dataset
# with open(os.path.join(REFORMED_DATASET_PATH, "train.json")) as f:
#     train = json.load(f) 
# with open(os.path.join(REFORMED_DATASET_PATH, "valid.json")) as f:
#     val = json.load(f)

In [5]:
# train_texts = [d['text'] for d in train]
# train_labels = [d['label'] for d in train]
# val_texts = [d['text'] for d in val]
# val_labels = [d['label'] for d in val]

In [6]:
# Encode dataset
# train_encodings = tokenizer(train_texts, truncation=True, padding=True)
# val_encodings = tokenizer(val_texts, truncation=True, padding=True)

In [7]:
# make dataset class
# train_dataset = IMDbDataset(train_encodings, train_labels)
# val_dataset = IMDbDataset(val_encodings, val_labels)

In [8]:
# train_loader = DataLoader(train_dataset, batch_size=1, shuffle=False)
# val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)

In [9]:
# Define tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [10]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = BertForSequenceClassification.from_pretrained(os.path.join(OUTPUT_PATH, 'epoch_2'))
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

## Gradient-based Masking

In [11]:
with open(os.path.join(REFORMED_DATASET_PATH, "train.json")) as f:
    data = json.load(f)

In [12]:
train_texts = [d['anchor_text'] for d in data][SPLIT_SAMPLES:SPLIT_SAMPLES*2]
train_labels = [d['label'] for d in data][SPLIT_SAMPLES:SPLIT_SAMPLES*2]

# Encode dataset
train_encodings = tokenizer(train_texts, truncation=True, padding=True)

# make dataset class
train_dataset = IMDbDataset(train_encodings, train_labels)

train_loader = DataLoader(train_dataset, batch_size=1, shuffle=False)

# Gradient Scoring Functions

In [13]:
# Compute gradient at BERT's position_embeddings (discard [cls] and [sep]/[pad])
# Only works for batch_size = 1    
def get_gradient_norms(batch):
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = batch['labels'].to(device)
    # For CrossEntropy Loss
    _, labels = torch.max(labels, dim=1)

    outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
    loss = outputs[0]
    # print(loss)
    loss.backward(retain_graph=True)
    torch.cuda.empty_cache()

    importances = torch.tensor([]).to(device)
    for pos_index, token_index in zip(range(1, len(input_ids[0])), input_ids[0][1:]):
        if token_index == tokenizer.sep_token_id:
            break

        importance = torch.norm(model._modules['bert']._modules['embeddings']._modules['position_embeddings'].weight.grad[pos_index], 2).float().detach()
        importances = torch.cat((importances, importance.unsqueeze(0)), dim=-1)

    # importances_list.append(importances)
    model._modules['bert']._modules['embeddings']._modules['position_embeddings'].weight.grad = None

    # return importances_list
    return importances

# Compute gradient at BERT's position_embeddings (discard [cls] and [sep]/[pad])
# Only works for batch_size = 1    
def get_token_gradient_norms(batch):
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = batch['labels'].to(device)
    # For CrossEntropy Loss
    _, labels = torch.max(labels, dim=1)

    outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
    loss = outputs[0]
    # print(loss)
    loss.backward(retain_graph=True)
    torch.cuda.empty_cache()

    importances = torch.tensor([]).to(device)
    for pos_index, token_index in zip(range(1, len(input_ids[0])), input_ids[0][1:]):
        if token_index == tokenizer.sep_token_id:
            break

        importance = torch.norm(model._modules['bert']._modules['embeddings']._modules['word_embeddings'].weight.grad[token_index], 2).float().detach()
        importances = torch.cat((importances, importance.unsqueeze(0)), dim=-1)

    # importances_list.append(importances)
    model._modules['bert']._modules['embeddings']._modules['word_embeddings'].weight.grad = None

    # return importances_list
    return importances

# Utility Functions

In [14]:
import random
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
import pickle
def visualize(words, masks):
    fig, ax = plt.subplots(figsize=(len(words), 1))
    plt.rc('xtick', labelsize=16)
    heatmap = sn.heatmap([masks], xticklabels=words, yticklabels=False, square=True, \
                         linewidths=0.1, cmap='coolwarm', center=0.5, vmin=0, vmax=1)
    plt.xticks(rotation=45)
    plt.show()
    

def mask_causal_words(tokens, importances, topk=1):
    causal_mask = [0 for _ in range(len(tokens))]
    topk_indices = np.argsort(importances)[::-1][:topk]
    for topk_idx in topk_indices:
#         print(topk_idx)
#         print(tokens[topk_idx])
        causal_mask[topk_idx] = 1
    
    return causal_mask

def compute_importances(data_loader, importance_function):
    all_importances = []
    for batch in tqdm(data_loader):
        importances = importance_function(batch)
        all_importances.append(importances)
    return all_importances


def compute_average_importance(data_loader, all_importances):
    all_averaged_importances = []
    importance_dict = dict()
    importance_dict_counter = dict()
    
    for importances, batch in tqdm(zip(all_importances, data_loader)):
        tokens = [x for x in batch['input_ids'][0][1:] if x not in [tokenizer.sep_token_id, tokenizer.pad_token_id]]
        
        for tok_imp, tok in zip(importances, tokens):
            if not tok in importance_dict.keys():
                importance_dict[tok.item()] = 0
                importance_dict_counter[tok.item()] = 0
            importance_dict[tok.item()] += tok_imp
            importance_dict_counter[tok.item()] += 1
            
    
    for importances, batch in tqdm(zip(all_importances, data_loader)):
        tokens = [x for x in batch['input_ids'][0][1:] if x not in [tokenizer.sep_token_id, tokenizer.pad_token_id]]
        averaged_importances = torch.tensor([importance_dict[x.item()]/importance_dict_counter[x.item()] for x in tokens])
        all_averaged_importances.append(averaged_importances)
    
    return all_averaged_importances


# Compute Gradient-based Causal Masks

In [15]:
def build_causal_mask_with_precomputed(data_loader, all_importances, sampling_ratio, augment_ratio):
    triplets = []
    error_cnt = 0
    for importances, batch in tqdm(zip(all_importances, data_loader)):
        tokens = torch.tensor([x for x in batch['input_ids'][0][1:] if x not in [tokenizer.sep_token_id, tokenizer.pad_token_id]])
        assert tokens.size() == importances.size()
        
        orig_sample = tokenizer.decode(tokens)
        causal_mask = mask_causal_words(tokens.cpu().numpy(), importances.cpu().numpy(), topk=sampling_ratio)
        # visualize(tokens, causal_mask)
        # print(causal_mask)
        
        if 1 not in causal_mask:
            # print(orig_sample[1], cf_sample[1])
            continue
        
        for _ in range(augment_ratio):
            # 모든 causal 단어를 mask, 모든 non-causal 단어를 mask
            if sampling_ratio is None:
                causal_masked_tokens = [tokens[i] if causal_mask[i] == 0 else tokenizer.mask_token_id for i in range(len(tokens))]
                noncausal_masked_tokens = [tokens[i] if causal_mask[i] == 1 else tokenizer.mask_token_id for i in range(len(tokens))]

            # sampling_ratio 갯수 (int) 만큼의 단어를 mask
            elif type(sampling_ratio) == int:
                causal_indices = np.where(np.array(causal_mask) == 1)[0]
                noncausal_indices = np.where(np.array(causal_mask) == 0)[0]

                # print(causal_indices)

                causal_mask_indices = np.random.choice(causal_indices, sampling_ratio)                    
                try:
                    noncausal_mask_indices = np.random.choice(noncausal_indices, max(1, min(sampling_ratio, len(noncausal_indices))))
                    #noncausal_mask_indices = np.random.choice(noncausal_indices, 1)
                except:
                    noncausal_mask_indices = np.random.choice(causal_indices, sampling_ratio)
                    error_cnt += 1

                causal_masked_tokens = [tokens[i] if i not in causal_mask_indices else tokenizer.mask_token_id for i in range(len(tokens))]
                noncausal_masked_tokens = [tokens[i] if i not in noncausal_mask_indices else tokenizer.mask_token_id for i in range(len(tokens))]
            
            # sampling_ratio 비율 (%) 만큼의 단어를 mask
            else:
                pass
                
            causal_masked_sample = tokenizer.decode(causal_masked_tokens)
            noncausal_masked_sample = tokenizer.decode(noncausal_masked_tokens)
            
            _, labels = torch.max(batch['labels'], dim=1)
            if labels[0] == 0: label = 'Negative'
            elif labels[0] == 1: label = 'Positive'
            triplets.append((label, orig_sample, causal_masked_sample, noncausal_masked_sample))
    print(f"Error Cnt: {error_cnt}")    
    return triplets



# Compute Propensity-based Causal Mask

In [16]:
def _TVD(orig_logits, cf_logits):
    return 0.5 * torch.cdist(orig_logits.unsqueeze(0), cf_logits.unsqueeze(0), p=1).squeeze().item()
"""For Debugging"""
"""
flipped_TVD = []
unflipped_TVD = []
def mask_softed_propensity_causal_words(tokens, batch, importances, topk=1):
    causal_mask = [0 for _ in range(len(tokens))]
    topk_indices = np.argsort(importances)[::-1]
    err_flag = False
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    orig_outputs = model(input_ids, attention_mask=attention_mask)
    _, orig_prediction = torch.max(orig_outputs[0], dim=1)
    for i, topk_idx in enumerate(topk_indices):
        masked_input_ids = input_ids.clone()
        masked_input_ids[0][topk_idx + 1] = tokenizer.mask_token_id
        masked_outputs = model(masked_input_ids, attention_mask=attention_mask)
        _, masked_prediction = torch.max(masked_outputs[0], dim=1)
        if orig_prediction != masked_prediction:
            #DEBUGGING
            flipped_TVD.append(_TVD(torch.softmax(orig_outputs[0], dim=-1), torch.softmax(masked_outputs[0], dim=-1)))
            #DEBUGGING END
            causal_mask[topk_idx] = 1
            break
        else:
            #DEBUGGING
            unflipped_TVD.append(_TVD(torch.softmax(orig_outputs[0], dim=-1), torch.softmax(masked_outputs[0], dim=-1)))
            #DEBUGGING END

    if 1 not in causal_mask:
        causal_mask[topk_indices[0]] = 1
        err_flag = True
    return causal_mask, err_flag 
"""
def mask_uniform_propensity_causal_words(tokens, batch, importances, topk=1):
    THRESHOLD = 0.1
    uniform_dist = torch.ones(1, 2) / 2.0
    uniform_dist = uniform_dist.to("cuda")
    causal_mask = [0 for _ in range(len(tokens))]
    topk_indices = np.argsort(importances)[::-1]
    err_flag = False
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    orig_outputs = model(input_ids, attention_mask=attention_mask)
    _, orig_prediction = torch.max(orig_outputs[0], dim=1)
    best_tvd = 0.
    best_idx = -1
    for i, topk_idx in enumerate(topk_indices):
        masked_input_ids = input_ids.clone()
        masked_input_ids[0][topk_idx + 1] = tokenizer.mask_token_id
        masked_outputs = model(masked_input_ids, attention_mask=attention_mask)
        _, masked_prediction = torch.max(masked_outputs[0], dim=1)
        orig_tvd = _TVD(torch.softmax(orig_outputs[0], dim=-1), uniform_dist)
        masked_tvd = _TVD(torch.softmax(masked_outputs[0], dim=-1), uniform_dist)
        
        # Use maximum value
        if orig_tvd > masked_tvd and abs(orig_tvd - masked_tvd) > best_tvd:
            causal_mask[best_idx] = 0
            causal_mask[topk_idx] = 1
            best_tvd = abs(orig_tvd - masked_tvd)
            best_idx = topk_idx
        else:
            continue
        """
        # Use Gradient Order
        if orig_tvd > masked_tvd:
            causal_mask[topk_idx] = 1
            break
        else:
            continue
        """    
        
    if 1 not in causal_mask:
        causal_mask[topk_indices[0]] = 1
        err_flag = True
    return causal_mask, err_flag, best_tvd

def mask_softed_propensity_causal_words(tokens, batch, importances, topk=1):
    causal_mask = [0 for _ in range(len(tokens))]
    topk_indices = np.argsort(importances)[::-1]
    err_flag = False
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    orig_outputs = model(input_ids, attention_mask=attention_mask)
    _, orig_prediction = torch.max(orig_outputs[0], dim=1)
    best_tvd = 0.
    best_idx = -1
    for i, topk_idx in enumerate(topk_indices):
        masked_input_ids = input_ids.clone()
        masked_input_ids[0][topk_idx + 1] = tokenizer.mask_token_id
        masked_outputs = model(masked_input_ids, attention_mask=attention_mask)
        _, masked_prediction = torch.max(masked_outputs[0], dim=1)
        tvd_value = _TVD(torch.softmax(orig_outputs[0], dim=-1), torch.softmax(masked_outputs[0], dim=-1))
        
        # Use Maximum  Value
        if tvd_value > best_tvd:
            causal_mask[best_idx] = 0
            causal_mask[topk_idx] = 1
            best_tvd = tvd_value
            best_idx = topk_idx
        else:
            continue
            
        """
        # Use Gradeint Order
        if orig_tvd > MIN_FLIPPED:
            causal_mask[topk_idx] = 1
            break
        else:
            continue
        """
        
    if 1 not in causal_mask:
        causal_mask[topk_indices[0]] = 1
        err_flag = True
    return causal_mask, err_flag, best_tvd

def mask_propensity_causal_words(tokens, batch, importances, topk=1):
    causal_mask = [0 for _ in range(len(tokens))]
    topk_indices = np.argsort(importances)[::-1]
    err_flag = False
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    orig_outputs = model(input_ids, attention_mask=attention_mask)
    _, orig_prediction = torch.max(orig_outputs[0], dim=1)
    for i, topk_idx in enumerate(topk_indices):
        masked_input_ids = input_ids.clone()
        masked_input_ids[0][topk_idx + 1] = tokenizer.mask_token_id
        masked_outputs = model(masked_input_ids, attention_mask=attention_mask)
        _, masked_prediction = torch.max(masked_outputs[0], dim=1)
        if orig_prediction != masked_prediction:
            causal_mask[topk_idx] = 1
            break
    if 1 not in causal_mask:
        causal_mask[topk_indices[0]] = 1
        err_flag = True
    return causal_mask, err_flag, 0

def build_propensity_causal_mask_with_precomputed(data_loader, all_importances, sampling_ratio, augment_ratio):
    triplets = []
    error_cnt = 0
    no_flip_cnt = 0
    no_flip_idx = []
    for importances, batch in tqdm(zip(all_importances, data_loader)):
        tokens = torch.tensor([x for x in batch['input_ids'][0][1:] if x not in [tokenizer.sep_token_id, tokenizer.pad_token_id]])
        assert tokens.size() == importances.size()
        
        orig_sample = tokenizer.decode(tokens)
        causal_mask, err_flag, maximum_score = mask_propensity_causal_words(tokens.cpu().numpy(), batch, importances.cpu().numpy(), topk=sampling_ratio)
        #causal_mask, err_flag, maximum_score = mask_softed_propensity_causal_words(tokens.cpu().numpy(), batch, importances.cpu().numpy(), topk=sampling_ratio)
        #causal_mask, err_flag, maximum_score = mask_uniform_propensity_causal_words(tokens.cpu().numpy(), batch, importances.cpu().numpy(), topk=sampling_ratio)
        no_flip_idx.append(err_flag)
        if err_flag:
            no_flip_cnt += 1
        # visualize(tokens, causal_mask)
        # print(causal_mask)
        
        if 1 not in causal_mask:
            # print(orig_sample[1], cf_sample[1])
            continue
        
        for _ in range(augment_ratio):
            # 모든 causal 단어를 mask, 모든 non-causal 단어를 mask
            if sampling_ratio is None:
                causal_masked_tokens = [tokens[i] if causal_mask[i] == 0 else tokenizer.mask_token_id for i in range(len(tokens))]
                noncausal_masked_tokens = [tokens[i] if causal_mask[i] == 1 else tokenizer.mask_token_id for i in range(len(tokens))]

            # sampling_ratio 갯수 (int) 만큼의 단어를 mask
            elif type(sampling_ratio) == int:
                causal_indices = np.where(np.array(causal_mask) == 1)[0]
                noncausal_indices = np.where(np.array(causal_mask) == 0)[0]

                # print(causal_indices)

                causal_mask_indices = np.random.choice(causal_indices, sampling_ratio)                    
                try:
                    noncausal_mask_indices = np.random.choice(noncausal_indices, max(1, min(sampling_ratio, len(noncausal_indices))))
                    #noncausal_mask_indices = np.random.choice(noncausal_indices, 1)
                except:
                    noncausal_mask_indices = np.random.choice(causal_indices, sampling_ratio)
                    error_cnt += 1

                causal_masked_tokens = [tokens[i] if i not in causal_mask_indices else tokenizer.mask_token_id for i in range(len(tokens))]
                noncausal_masked_tokens = [tokens[i] if i not in noncausal_mask_indices else tokenizer.mask_token_id for i in range(len(tokens))]
            
            # sampling_ratio 비율 (%) 만큼의 단어를 mask
            else:
                pass
                
            causal_masked_sample = tokenizer.decode(causal_masked_tokens)
            noncausal_masked_sample = tokenizer.decode(noncausal_masked_tokens)
            
            _, labels = torch.max(batch['labels'], dim=1)
            if labels[0] == 0: label = 'Negative'
            elif labels[0] == 1: label = 'Positive'
            triplets.append((label, orig_sample, causal_masked_sample, noncausal_masked_sample, err_flag, maximum_score))
    print(f"Error Cnt: {error_cnt}")    
    print(f"No Flip Cnt: {no_flip_cnt}")    
    return triplets, no_flip_idx


# Compute OR Load Gradient Importance

In [17]:
if not os.path.exists(PICKLE_PATH):
    os.makedirs(PICKLE_PATH)

#all_importance = compute_importances(train_loader, get_gradient_norms)
#with open("dataset/SST-2/cf_augmented_examples/gradient_importance.pickle", 'wb') as f:
#    pickle.dump(all_importance, f)

if os.path.exists(os.path.join(PICKLE_PATH, "gradient_importance.pickle")):
    with open(os.path.join(PICKLE_PATH, "gradient_importance.pickle"), 'rb') as f:
        all_importance = pickle.load(f)
else:
    all_importance = compute_importances(train_loader, get_gradient_norms)
    with open(os.path.join(PICKLE_PATH, "gradient_importance.pickle"), 'wb') as f:
        pickle.dump(all_importance, f)

# Get Average Importance

In [18]:
if os.path.exists(os.path.join(PICKLE_PATH, "gradient_averaged_importance.pickle")):
    with open(os.path.join(PICKLE_PATH, "gradient_averaged_importance.pickle"), 'rb') as f:
        averaged_all_importance = pickle.load(f)
else:
    raise ValueError("Please run averaged importance on original code.")

averaged_all_importance = averaged_all_importance[SPLIT_SAMPLES:SPLIT_SAMPLES*2]

# Generate Triplets

In [ ]:
sampling_ratio = 1
augment_ratio = 1
triplets_train, no_flip_idx_train = build_propensity_causal_mask_with_precomputed(train_loader, averaged_all_importance, sampling_ratio=sampling_ratio, augment_ratio=augment_ratio)

In [ ]:
with open(os.path.join(PICKLE_PATH, "triplets_automated_averaged_gradient_propensity_flip_sampling{}_augmenting{}_train_cuda1.pickle".format(sampling_ratio, augment_ratio)), "wb") as fp:
    pickle.dump(triplets_train, fp)

# Cut with Empirically  Setted Threshold

In [68]:
THRESHOLD = 0.0001
threshold_cnt = 0
threshold_triplets_train = []
for t in triplets_train:
    t = list(t)
    if t[-1] < THRESHOLD:
        t[-2] = True
        threshold_cnt += 1
    t = tuple(t)
    threshold_triplets_train.append(t)
print(threshold_cnt)

6862


In [69]:
with open(os.path.join(PICKLE_PATH, "triplets_automated_averaged_gradient_propensity_TVD_uniform_thres_00001_sampling{}_augmenting{}_train.pickle".format(sampling_ratio, augment_ratio)), "wb") as fp:
    pickle.dump(threshold_triplets_train, fp)

In [21]:
###For Generating Optionally

#for sampling_ratio in [1, 2, 3, 4, 5]:
for sampling_ratio in [1]:
    for augment_ratio in [1]:
        triplets_train, no_flip_idx_train = build_propensity_causal_mask_with_precomputed(train_loader, averaged_all_importance, sampling_ratio=sampling_ratio, augment_ratio=augment_ratio)
        with open(os.path.join(PICKLE_PATH, "triplets_automated_averaged_gradient_propensity_sampling{}_augmenting{}_train.pickle".format(sampling_ratio, augment_ratio)), "wb") as fp:
            pickle.dump(triplets_train, fp)

KeyboardInterrupt: 

In [22]:
MIN_FLIPPED = 0.007849693298339844 #

In [94]:
c = sorted(unflipped_TVD_logit)
print(min(flipped_TVD_logit))
print(c[int(len(c) * 0.90)])

0.015699736773967743
0.5579570531845093


# Qualitative Test

In [ ]:
#SHELL FOR QUALITATIVE: WILL BE REMOVED\
sampling_ratio = 1
augment_ratio = 1
triplets_train, no_flip_idx_train = build_propensity_causal_mask_with_precomputed(train_loader, averaged_all_importance, sampling_ratio=None, augment_ratio=augment_ratio)

In [32]:
qual_triplets = [tt for tt in triplets_train if tt[2].count("[MASK]") > 1]

In [93]:
qual_triplets[37]

('Positive',
 'defuses this provocative theme by submerging it in a hoary love triangle.',
 'def [MASK] this [MASK] theme by submerging it in a hoary love triangle.',
 '[MASK]uses [MASK] provocative [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK]',
 False)

In [75]:
for i, tt in enumerate(triplets_train):
    if tt[1] == 'commercialism all in the same movie... without neglecting character development for even one minute':
        print(i)

1590


In [55]:
with open(os.path.join(PICKLE_PATH, "triplets_automated_averaged_gradient_propensity_sampling1_augmenting1_train.pickle".format(sampling_ratio, augment_ratio)), "rb") as fp:
            data_b = pickle.load(fp)
        
with open(os.path.join(PICKLE_PATH, "triplets_automated_averaged_gradient_sampling1_augmenting1_train.pickle".format(sampling_ratio, augment_ratio)), "rb") as fp:
            data_a = pickle.load(fp)

In [57]:
cnt = 0
diff_idxes = []
for i, (a, b) in enumerate(zip(data_a, data_b)):
    if a[2] != b[2]:
        diff_idxes.append(i)
        cnt += 1

In [ ]:
tmp_i = 3
print("[ORIG]:    " + data_a[diff_idxes[tmp_i]][1])
print()
print("[GRAD]:    " + data_a[diff_idxes[tmp_i]][2])
print()
print("[PROP]:    " + data_b[diff_idxes[tmp_i]][2])

In [81]:
torch.set_printoptions(precision=4,sci_mode=False)
print(tokenizer.tokenize('commercialism all in the same movie... without neglecting character development for even one minute'))
print(averaged_all_importance[1590])

['commercial', '##ism', 'all', 'in', 'the', 'same', 'movie', '.', '.', '.', 'without', 'neglect', '##ing', 'character', 'development', 'for', 'even', 'one', 'minute']
tensor([    0.0017,     0.0001,     0.0004,     0.0002,     0.0013,     0.0038,
            0.0000,     0.0015,     0.0015,     0.0015,     0.0001,     0.0007,
            0.0005,     0.0002,     0.0009,     0.0019,     0.0001,     0.0002,
            0.0175])


In [82]:
data_a[1590]

('Positive',
 'commercialism all in the same movie... without neglecting character development for even one minute',
 'commercialism all in the same movie... without neglecting character development for even one [MASK]',
 'commercialism [MASK] in the same movie... without neglecting character development for even one minute')

In [62]:
9719 / - (53349 - 67349)

0.6942142857142857